<a href="https://colab.research.google.com/github/huishingchong/agile_llm/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Pipeline

## Set up
### Import Packages and API keys

In [1]:
!pip install transformers datasets torch torchvision torchaudio langchain-community faiss-cpu sentence-transformers langchain gradio mlflow evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
# get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token
from getpass import getpass
import os

HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN


··········


In [ ]:
from getpass import getpass
from dotenv import load_dotenv
import os
from pathlib import Path

env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

huggingface_api_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

if not huggingface_api_token:
    huggingface_api_token = getpass("Enter your Hugging Face Hub API token: ")

### Load Model

In [4]:
from langchain_community.llms import HuggingFaceEndpoint
model_name = "tiiuae/falcon-7b-instruct"
llm = HuggingFaceEndpoint(
    repo_id=model_name,
    model=model_name,
    task="text-generation",
    temperature=0.5,
    # max_length:1024,
    max_new_tokens=200
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## RAG from synthetic data set (skills build)

### Set up documents, embeddings and retrievers

In [5]:
# Use langchain packages to help with implementing retrieval augmentation generation
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer

In [7]:
# Load data for retrieval from a csv file
sb_loader = CSVLoader(file_path="skills_build.csv")
sb_documents = sb_loader.load()

# Split documents into chunks
text_split = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
sb_splitted_documents = text_split.split_documents(sb_documents)

In [8]:
modelPath = "sentence-transformers/gtr-t5-base" # Using t5 sentence transformer model to generate embeddings
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings': True} # Normalizing embeddings may help improve similarity metrics by ensuring that embeddings magnitude does not affect the similarity scores

# Initialise an instance of HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [9]:
# Create a FAISS Index with specified document and embeddings
sb_db = FAISS.from_documents(sb_splitted_documents, embedding=embeddings)

In [ ]:
# check that retriever works
query_example = "What is IBM SkillsBuild credentials?"
sb_retriever = sb_db.as_retriever()
sb_docs = sb_retriever.get_relevant_documents(query_example)
sb_docs

[Document(page_content="IBM SkillsBuild: Digital credentials\nDescription: Digital credentials from IBM SkillsBuild are secure, web-enabled credentials that contain granular, verified information that employers can use to evaluate an individual's potential. Digital credential requirements can include online learning activities, quizzes or exams, experience that requires advisor review, or even interviews. Complete the required activities, accept the credential, and choose to share your achievements to your resume, LinkedIn, and other social media.", metadata={'source': 'skills_build.csv', 'row': 7}),
 Document(page_content='IBM SkillsBuild: broader initiative\nDescription: IBM SkillsBuild is a free education program focused on underrepresented communities that helps adult learners, high school and university students, and faculty develop valuable new skills and access career opportunities. The program includes an online platform complemented by customised practical learning experiences

In [ ]:
for doc in sb_docs:
    print(f"\nContent: {doc.page_content}, Metadata: {doc.metadata}")

Evaluate retrieval

In [ ]:
sb_docs_and_scores = sb_db.similarity_search_with_score(query_example)
print(sb_docs_and_scores)

In [ ]:
for doc, score in sb_docs_and_scores:
    print(f"\nContent: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

### RetrievalQA

In [10]:
from langchain.prompts import PromptTemplate

# template = """Use the following context to answer the question at the end.
# If you don't know the answer, please think rationally and answer from your own knowledge base.
# Context: {context}

# Question: {question}
# """
template = """Use the following context to answer the question at the end.
If you don't know the answer, please think rationally and answer from your own knowledge base.
Context: {context}

Question: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])

In [11]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain

chain_type_kwargs = {"prompt": QA_CHAIN_PROMPT}
sb_qa = RetrievalQA.from_chain_type(llm=llm,
                                 retriever=sb_db.as_retriever(),
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs, verbose=True)


In [ ]:
sb_response = sb_qa.invoke({"query": "What is the responsibility of the Penetration Tester?"})
print(sb_response["result"])

In [ ]:
# See the source documents
print(sb_response.get("source_documents", []))

In [ ]:
import gradio as gr

def chat_interface(textbox, chat):
    input_dict = {'query': textbox}
    result = sb_qa.invoke(input_dict)
    print(result)
    text = result['result']
    return text

gr.ChatInterface(
    fn=chat_interface,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="Chatbot",
    description="Ask Chatbot any question",
    theme="soft",
    cache_examples=False,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch(debug=True)

## Evaluation

In [12]:
!pip install torch tiktoken evaluate mlflow bert_score evaluate textstat


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.5 MB/s eta 0:00:00


Prepare data set

In [13]:
import pandas as pd
data_skills_build = pd.DataFrame({
    "question": [
        "How does IBM SkillsBuild help individuals develop their technical skills and advance their careers?",
        "Can you describe any collaborative or interactive features available on the IBM SkillsBuild platform for engaging with peers or instructors?",
        "Can you explain the different learning paths available on IBM SkillsBuild platform?",
        "What resources does IBM SkillsBuild offer to different age groups and how does it support their learning and professional development goals?",
        "How does IBM SkillsBuild support personalized learning experiences for users?",
        "What resources does IBM SkillsBuild offer to help users prepare for certification exams in various technology domains?",
        "Can you explain the process of navigating through courses and learning paths on the IBM SkillsBuild platform?",
        "Can users access IBM SkillsBuild on mobile devices?",
        "What types of support resources are available to users who encounter technical difficulties or need assistance while using the platform?",
        "What is the badge program in IBM SkillsBuild?"
    ],
    "ground_truth": [
        "With IBM SkillsBuild, individuals can develop their technical skills and advance their careers through a comprehensive suite of features and resources tailored to their needs. The platform offers a diverse range of learning opportunities, including modules on leadership, communication, programming, problem-solving, and writing skills, all of which are crucial for success in today's job market. By providing access to high-quality educational content and interactive learning experiences, IBM SkillsBuild empowers users to acquire new skills, deepen their expertise, and stay competitive in their respective fields. Additionally, the platform offers digital credentials that allow users to showcase their achievements and qualifications to potential employers, enhancing their credibility and marketability in the job market. Overall, IBM SkillsBuild serves as a valuable resource for individuals looking to upskill, reskill, or transition into new career paths, enabling them to pursue their professional goals with confidence and competence.",
        "IBM SkillsBuild offers a comprehensive learning experience that goes beyond traditional online courses. With discussion forums, live webinars and virtual events, and a mentorship program, learners have ample opportunities to engage with peers and experts, enhancing their educational journey. Discussion forums provide a platform for learners to ask questions, share insights, and collaborate with others, fostering a sense of community and facilitating peer learning. Live webinars and virtual events bring industry experts and instructors directly to learners, covering diverse topics and providing valuable insights and opportunities for interaction. The mentorship program pairs learners with experienced professionals who guide them through real-world projects, imparting practical skills and knowledge. Additionally, learners benefit from premium content in 22 languages, access to the same certifications as IBM employees, and connections to career opportunities, including job fairs, workshops, and internships. Overall, IBM SkillsBuild equips learners with valuable skills while connecting them to a network of peers, experts, and career opportunities.",
        "IBM SkillsBuild offers diverse paths for individuals to enhance their skills and pursue career goals. These paths include technical skills development in areas like programming, cloud computing, and cybersecurity, as well as soft skills enhancement such as communication and leadership. Career advancement tracks are available for fields like software development and digital marketing, while industry-specific learning paths cater to sectors like healthcare and finance. Additionally, individuals can obtain specialized certifications to validate their expertise in specific technologies or domains. Overall, IBM SkillsBuild provides a comprehensive platform for professional development, empowering learners to acquire new skills, advance their careers, and achieve their aspirations.",
        "Adult Learners is designed for learners (age 18+) seeking employment in the immediate future. It provides adult learners with free job role preparation specifically focused on entry-level tech roles to help them gain meaningful employment, while earning credentials. High School Students is designed for learners at the career exploration stage (aged 13 to 18). It’s designed to help students begin their career exploration with free digital learning on cutting-edge technology and workplace skills. Academia is designed for university students and faculty. Students from IBM’s partner Universities can earn digital credentials, access software, and explore a library of university guest lectures across data science, artificial intelligence, security and more. Learners also have access to expert conversations from IBM volunteers, access to IBM software, and project-based learning (practical exercises to apply learning they have acquired, including labs using IBM technology). Software Downloads is an element within the IBM SkillsBuild program designed to provide learners and educators at degree-granting accredited academic institutions with access to select IBM assets at no-charge for classroom (teaching and learning) and non-commercial research purposes at the academic institution.",
        "IBM SkillsBuild supports personalized learning experiences for users through several key features and capabilities: 1. Skill Assessment: The platform offers skill assessments to evaluate users' current proficiency levels in various domains, including technology, business, and digital literacy. These assessments help identify users' strengths and areas for improvement, enabling personalized learning paths. 2. Personalized Learning Paths: Based on the results of skill assessments and users' career goals, IBM SkillsBuild creates personalized learning paths tailored to each individual. These learning paths consist of curated learning modules, courses, and resources designed to address users' specific skill gaps and career objectives. 3. Adaptive Learning Content: The platform leverages adaptive learning technologies to dynamically adjust the difficulty and content of learning materials based on users' progress and performance. This ensures that users receive content that is appropriate for their skill levels and learning pace, optimizing the learning experience. 4. Recommendation Engine: IBM SkillsBuild employs a recommendation engine that suggests relevant courses, learning resources, and learning activities based on users' interests, preferences, and learning history. This personalized recommendation system helps users discover new topics, deepen their knowledge, and stay engaged with the learning process. 5. Interactive Learning Features: The platform offers interactive learning features such as quizzes, simulations, hands-on labs, and projects to enhance user engagement and retention. These interactive elements provide users with opportunities to apply their knowledge, practice skills, and receive immediate feedback, fostering active learning and skill development. 6. Progress Tracking and Feedback: IBM SkillsBuild enables users to track their learning progress, monitor their skill development, and receive feedback on their performance. Users can view their achievements, completion certificates, and skill badges, motivating them to continue learning and advancing their careers. Overall, IBM SkillsBuild's personalized learning approach empowers users to acquire new skills, gain valuable insights, and achieve their professional goals in a flexible and adaptive learning environment.",
        "IBM SkillsBuild offers a comprehensive range of resources to assist users in preparing for certification exams across various technology domains. These resources typically include: \nCourses and Learning Paths: SkillsBuild provides structured learning paths and courses designed to cover the topics and skills required for specific certification exams. These courses are often developed in collaboration with industry experts and align with the exam objectives. \nHands-on Labs and Exercises: Users can access hands-on labs and exercises to gain practical experience and reinforce theoretical concepts covered in the courses. These labs often simulate real-world scenarios and provide a sandbox environment for experimentation.\nPractice Tests and Assessments: SkillsBuild offers practice tests and assessments that mirror the format and difficulty level of the actual certification exams. These tests help users gauge their readiness and identify areas where they need to focus additional study.\nInteractive Tutorials and Videos: Interactive tutorials and instructional videos supplement the learning experience by providing visual explanations, demonstrations, and walkthroughs of key concepts and techniques.\nCommunity Forums and Support: Users can engage with a community of peers and experts through forums, discussion boards, and social media channels. This allows them to seek help, share insights, and collaborate with others who are also preparing for certification exams.",
        "The IBM SkillsBuild platform offers users a streamlined process for navigating through courses and learning paths. Starting from the dashboard, users can easily browse the course catalog, explore curated learning paths, and enroll in relevant content. Each course or learning path provides detailed information, including descriptions, objectives, and instructor details, allowing users to make informed decisions. Progress tracking features keep users updated on their learning journey, while interactive elements such as quizzes and hands-on labs enhance the learning experience. Upon completion, users receive certificates or digital badges to showcase their achievements, providing tangible evidence of their newfound skills. Overall, IBM SkillsBuild empowers users to pursue their learning goals effectively, with a user-friendly interface and comprehensive support throughout the learning process.",
        "Yes, users can access IBM SkillsBuild on mobile devices through the SkillsBuild App, which is designed for use with the SkillsBuild for Students website and its associated applications.",
        "IBM SkillsBuild offers robust support resources to assist users encountering technical difficulties or seeking guidance while navigating the platform. First, users can access an extensive online Help Center, providing a wealth of information including FAQs, troubleshooting guides, and step-by-step tutorials. Additionally, community forums serve as collaborative spaces where users can pose questions, share insights, and receive assistance from both peers and platform experts. Furthermore, users have direct access to customer support channels, enabling them to seek personalized assistance for technical issues, account-related queries, or other concerns via email, live chat, or phone support.",
        "A digital badge is a cross-industry digital recognition of technical skills which can be shared on your social and professional networking sites, as well as your digital signature.  IBM is supporting this program, based on open standards, through the creation of a wide variety of badges covering a multitude of technical and professional areas."
    ]
})

retrieval_eval = pd.DataFrame({
    "question": [
        "What is the goal of IBM SkillsBuild?",
        "What are the courses available at IBM SkillsBuild?",
        "What is IBM SkillsBuild Software downloads?",
        "What learning paths do IBM SkillsBuild offer to different age groups and how does it support their learning and professional development goals?",
        "How does IBM SkillsBuild support personalized learning experiences for users?",
        "What can learners do with earned digital credentials?",
        "What is a Credly account in IBM SkillsBuild?",
        "What can educators do on IBM SkillsBuild?",
        "What are the specific tools educators can access on IBM SkillsBuild?",
        "What are the specific tools or resources that college students enrolled on IBM SkillsBuild can access to enhance their learning?"
    ],
    "ground_truth": [
        "The goal of IBM SkillsBuild is to address the growing demand for essential technical and professional skills in the face of digital acceleration. It aims to empower learners from underrepresented communities, including adult learners, high school and university students, and faculty, by providing them with access to free education programs. These programs are designed to help individuals develop valuable new skills and access career opportunities. IBM SkillsBuild offers an online platform supplemented by customized practical learning experiences tailored to meet the evolving needs of learners as they progress through their education and career journeys. This initiative is carried out in collaboration with a network of esteemed education partners, including public high schools, non-profit organizations, governments, and corporations, to ensure widespread access and impact.",
        "IBM SkillsBuild offers a comprehensive range of technical and soft skills to prepare individuals for the evolving job market. On the technical front, learners can delve into cutting-edge areas such as AI, Blockchain, Cloud Computing, Cybersecurity, Data Science, Quantum Computing, and Web Development, among others. Meanwhile, the platform also focuses on cultivating essential workplace skills like Agile methodologies, Design Thinking, Professional skills, Job Readiness, and more. With this diverse array of offerings, IBM SkillsBuild equips learners with the expertise and adaptability needed in today's workplace.",
        "IBM SkillsBuild Software downloads is an element within the IBM SkillsBuild program designed to provide learners and educators at degree-granting accredited academic institutions with access to select IBM assets at no-charge for classroom (teaching and learning) and non-commercial research purposes at the academic institution.",
        "Adult Learners is designed for learners (age 18+) seeking employment in the immediate future. It provides adult learners with free job role preparation specifically focused on entry-level tech roles to help them gain meaningful employment, while earning credentials. High School Students is designed for learners at the career exploration stage (aged 13 to 18). It’s designed to help students begin their career exploration with free digital learning on cutting-edge technology and workplace skills. Academia is designed for university students and faculty. Students from IBM’s partner Universities can earn digital credentials, access software, and explore a library of university guest lectures across data science, artificial intelligence, security and more. Learners also have access to expert conversations from IBM volunteers, access to IBM software, and project-based learning (practical exercises to apply learning they have acquired, including labs using IBM technology). Software Downloads is an element within the IBM SkillsBuild program designed to provide learners and educators at degree-granting accredited academic institutions with access to select IBM assets at no-charge for classroom (teaching and learning) and non-commercial research purposes at the academic institution.",
        "The customized practical learning experiences provided by IBM SkillsBuild are designed to empower learners with valuable skills and opportunities for career advancement. Through project-based learning, participants can apply their newfound knowledge in practical exercises and labs using IBM technology. Additionally, insights from IBM Mentors offer guidance on technical expertise, business acumen, and collaboration skills. Learners have access to premium content, including online courses in 22 languages and certifications equivalent to those earned by IBM employees. Moreover, the platform connects individuals with career opportunities such as job fairs, workshops, and seminars, providing support with job interviews, professional portfolio creation, and internship opportunities, ultimately helping learners navigate their educational and career journeys with confidence.",
        "Digital credentials from IBM SkillsBuild can be shared on the learner's resume, LinkedIn, and other social media for peers and employers to see.",
        "Your Credly account is a repository to manage your digital credentials. It's where you claim (accept), store, and transmit your credentials. You can manage which credentials you want to display in your Credly account settings. To participate in the digital credential program, you need to create a Credly account. To accept or manage credentials, create or sign in to your Credly account. In your profile settings, make sure that the email address you use for your IBM SkillsBuild account is registered in your Credly account. To receive emails regarding issued credentials, turn on User Transactional e-mails. If you have an existing Credly account registered under a IBM SkillsBuild email, you can add your other email address to your account and make it the primary one.",
        "Educators can leverage IBM SkillsBuild to access a wealth of free learning resources tailored for their immediate needs. By signing up individually, educators gain access to a diverse array of materials designed to enhance their teaching methodologies and support student learning outcomes. Additionally, educators have the option to register their entire class or organization, unlocking full administrative access and additional features. This includes the ability to monitor students' learning progress, track course completions and monitor hours completed. Whether exploring resources as an individual or registering a class or organization, IBM SkillsBuild provides educators with valuable tools and support to enhance the educational experience and empower learners for success in the digital age.",
        "There is the Teacher toolkit, Career-readiness toolkit, and Curriculumn enhancement.",
        "College students accessing IBM SkillsBuild have a wide range of software tools at their disposal to enhance their learning and skill development. These include IBM Cloud, a comprehensive cloud platform spanning public, private, and hybrid environments, providing hands-on experience with cloud technologies. Additionally, students can utilize tools like ILOG CPLEX for analytical decision support, SPSS Modeller for advanced statistical analysis, and IBM Cognos Analytics for smarter analytics and decision-making. For cybersecurity education, students can access QRadar SIEM for threat detection and response. IBM SkillsBuild also offers educational resources on DB2 database management and various digital credentials to showcase skills acquired through courses. Furthermore, students can benefit from guest lectures by industry experts, further enriching their knowledge and learning experience. Overall, IBM SkillsBuild provides college students with valuable tools, resources, and opportunities to develop essential skills and prepare for future career paths in the digital age."
    ]
})

In [14]:
# Helper function: To prevent having to run predictions again, save output to the dataset as a new column 'predictions'
import datasets

def chain_predictions_from_data(llm_chain, eval_data):
  data = eval_data.copy()
  predictions = []  # List to store predictions for all questions
  source_documents = []
  for question in data['question']:
    # Invoke the llm_chain model with the current question
    input_dict = {'query': question}
    response = llm_chain.invoke(input_dict)
    generated_text = response['result']
    predictions.append(generated_text)  # Append the generated text to the predictions list
    print(question)
    print(generated_text)
    documents = response.get("source_documents", [])
    concatenated_page_content = ""
    for document in documents:
    # Append the page_content of each Document to the page_contents array
      concatenated_page_content += document.page_content + "\n"
    source_documents.append(concatenated_page_content)
  data['source_documents'] = source_documents
  data['predictions'] = predictions  # Assign the predictions list to a new column
  return data

In [15]:
rag_data_skills_build = chain_predictions_from_data(sb_qa, data_skills_build)
os.makedirs("skillsbuild", exist_ok=True)
rag_data_skills_build.to_csv("skillsbuild/rag.csv", index=False)



> Entering new RetrievalQA chain...

> Finished chain.
What is the goal of IBM SkillsBuild?
Answer: The goal of IBM SkillsBuild is to help individuals gain the necessary skills to succeed in the modern workforce by providing them with access to free digital learning resources, mentorship, and job opportunities.


> Entering new RetrievalQA chain...

> Finished chain.
What are the courses available at IBM SkillsBuild?
Answer: IBM SkillsBuild provides a wide range of courses, including AI, Blockchain, Cloud Computing, Cybersecurity, Data science, Quantum computing, Web development, and more.


> Entering new RetrievalQA chain...

> Finished chain.
What is IBM SkillsBuild Software downloads?
Answer: IBM SkillsBuild Software downloads is an element within the IBM SkillsBuild program designed to provide learners and educators at degree-granting accredited academic institutions with access to select IBM assets at no-charge for classroom (teaching and learning) and non-commercial research p

In [16]:
rag_data_skills_build

,question,ground_truth,source_documents,predictions
0,What is the goal of IBM SkillsBuild?,The goal of IBM SkillsBuild is to address the ...,IBM SkillsBuild: broader initiative\nDescripti...,Answer: The goal of IBM SkillsBuild is to help...
1,What are the courses available at IBM SkillsBu...,IBM SkillsBuild offers a comprehensive range o...,IBM SkillsBuild: Course catalogue\nDescription...,Answer: IBM SkillsBuild provides a wide range ...
2,What is IBM SkillsBuild Software downloads?,IBM SkillsBuild Software downloads is an eleme...,IBM SkillsBuild: Software downloads\nDescripti...,Answer: IBM SkillsBuild Software downloads is ...
3,What learning paths do IBM SkillsBuild offer t...,Adult Learners is designed for learners (age 1...,IBM SkillsBuild: For High school learners\nDes...,IBM SkillsBuild offers learning paths to diffe...
4,How does IBM SkillsBuild support personalized ...,The customized practical learning experiences ...,IBM SkillsBuild: broader initiative\nDescripti...,Answer: IBM SkillsBuild supports personalized ...
5,What can learners do with earned digital crede...,Digital credentials from IBM SkillsBuild can b...,IBM SkillsBuild: Digital credentials\nDescript...,Answer: Learners can use their digital credent...
6,What is a Credly account in IBM SkillsBuild?,Your Credly account is a repository to manage ...,IBM SkillsBuild: Digital credentials\nDescript...,A Credly account is an account that can be use...
7,What can educators do on IBM SkillsBuild?,Educators can leverage IBM SkillsBuild to acce...,IBM SkillsBuild: broader initiative\nDescripti...,Answer: Educators can access free digital lear...
8,What are the specific tools educators can acce...,"There is the Teacher toolkit, Career-readiness...",IBM SkillsBuild: Software downloads\nDescripti...,Answer: Educators on IBM SkillsBuild can acces...
9,What are the specific tools or resources that ...,College students accessing IBM SkillsBuild hav...,IBM SkillsBuild: broader initiative\nDescripti...,The specific tools and resources that college ...


Generate bert score results table for the data set

In [17]:
from evaluate import load

def generate_bert_results_table(eval_data):
    ref_texts = eval_data['ground_truth']
    predictions = eval_data['predictions']

    # Compute BERTScore
    bertscore = load("bertscore")
    results = bertscore.compute(predictions=predictions, references=ref_texts, model_type="distilbert-base-uncased")

    # Create DataFrame from BERTScore results
    bert_results_table = pd.DataFrame(results, index=range(0, len(results['precision'])))
    bert_results_table['question'] = eval_data['question']

    bert_results_table.drop(columns=['hashcode'], inplace=True)
    bert_results_table = bert_results_table[['question', 'precision', 'recall', 'f1']]

    return bert_results_table

In [18]:
generate_bert_results_table(rag_data_skills_build)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

,question,precision,recall,f1
0,What is the goal of IBM SkillsBuild?,0.885881,0.792965,0.836851
1,What are the courses available at IBM SkillsBu...,0.939811,0.815476,0.873240
2,What is IBM SkillsBuild Software downloads?,0.988668,0.997640,0.993133
3,What learning paths do IBM SkillsBuild offer t...,0.837632,0.772414,0.803702
4,How does IBM SkillsBuild support personalized ...,0.852409,0.814008,0.832766
5,What can learners do with earned digital crede...,0.796053,0.810355,0.803141
6,What is a Credly account in IBM SkillsBuild?,0.826150,0.769402,0.796767
7,What can educators do on IBM SkillsBuild?,0.830273,0.790019,0.809646
8,What are the specific tools educators can acce...,0.724205,0.804344,0.762174
9,What are the specific tools or resources that ...,0.803478,0.815081,0.809238


UniEval multi-dimensional metrics and factual consistency

In [19]:
!git clone https://github.com/maszhongming/UniEval.git
%cd UniEval
!pip install -r requirements.txt

Cloning into 'UniEval'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 91 (delta 13), reused 5 (delta 5), pack-reused 65
Receiving objects: 100% (91/91), 1.97 MiB | 14.85 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/UniEval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/

In [23]:
import torch
import nltk
from utils import convert_to_json
from metric.evaluator import get_evaluator

nltk.download('punkt')
torch.cuda.is_available()

True

In [22]:
# Factual consistency score
task = 'fact'

src_list = rag_data_skills_build["source_documents"]
output_list = rag_data_skills_build["predictions"]

# Prepare data for pre-trained evaluators
data = convert_to_json(output_list=output_list, src_list=src_list)
# Initialize evaluator for a specific task
evaluator = get_evaluator(task)
# Get factual consistency scores
eval_scores = evaluator.evaluate(data, print_result=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Evaluating consistency of 10 samples !!!


100%|██████████| 4/4 [00:02<00:00,  1.36it/s]


Evaluation scores are shown below:
+-------------+----------+
|  Dimensions |  Score   |
+-------------+----------+
| consistency | 0.711873 |
+-------------+----------+


In [25]:

task = 'dialogue'

# a list of dialogue histories
src_list = rag_data_skills_build["question"]
# a list of additional context that should be included into the generated response
context_list = rag_data_skills_build["source_documents"]
# a list of model outputs to be evaluated
output_list = rag_data_skills_build["predictions"]

data = convert_to_json(output_list=output_list,
                       src_list=src_list, context_list=context_list)
evaluator = get_evaluator(task)
eval_scores = evaluator.evaluate(data, print_result=True)

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

...

Evaluating groundedness of 10 samples !!!


100%|██████████| 2/2 [00:01<00:00,  1.50it/s]


Evaluating understandability of 10 samples !!!


100%|██████████| 2/2 [00:00<00:00,  5.97it/s]


Evaluation scores are shown below:
+-------------------+----------+
|     Dimensions    |  Score   |
+-------------------+----------+
|    naturalness    | 0.813156 |
|     coherence     |  0.9983  |
|    engagingness   | 2.384515 |
|    groundedness   | 0.923365 |
| understandability | 0.806496 |
|      overall      | 1.185166 |
+-------------------+----------+


MLFlow: Toxicity and Readability metrics

In [ ]:
# to connect to mlflow server, run in terminal: mlflow ui

In [26]:
import mlflow
from mlflow.models import infer_signature
import pandas as pd

input_columns = [{"question": "string"}]
output_columns = [{"text": "string"}]
signature = infer_signature(input_columns, output_columns)

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("/falcon-evaluation")

# eval_data: dataframe with questions, ground_truth and predictions
def mlflow_evaluation(eval_data):
  with mlflow.start_run() as run:
    mlflow_results = mlflow.evaluate(
        data=eval_data,
        targets="ground_truth",
        predictions="predictions",
        model_type="question-answering",
    )
    # print(f"See aggregated evaluation results below: \n{mlflow_results.metrics}")

    eval_table = mlflow_results.tables["eval_results_table"]
    print(f"See evaluation table below: \n{eval_table}")

2024/03/09 22:38:27 INFO mlflow.tracking.fluent: Experiment with name '/falcon-evaluation' does not exist. Creating a new experiment.


In [27]:
mlflow_evaluation(rag_data_skills_build)

2024/03/09 22:38:31 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/03/09 22:38:31 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

See evaluation table below: 
                                            question  \
0               What is the goal of IBM SkillsBuild?   
1  What are the courses available at IBM SkillsBu...   
2        What is IBM SkillsBuild Software downloads?   
3  What learning paths do IBM SkillsBuild offer t...   
4  How does IBM SkillsBuild support personalized ...   
5  What can learners do with earned digital crede...   
6       What is a Credly account in IBM SkillsBuild?   
7          What can educators do on IBM SkillsBuild?   
8  What are the specific tools educators can acce...   
9  What are the specific tools or resources that ...   

                                    source_documents  \
0  IBM SkillsBuild: broader initiative\nDescripti...   
1  IBM SkillsBuild: Course catalogue\nDescription...   
2  IBM SkillsBuild: Software downloads\nDescripti...   
3  IBM SkillsBuild: For High school learners\nDes...   
4  IBM SkillsBuild: broader initiative\nDescripti...   
5  IBM SkillsBuild

### Hardware specs

In [28]:
#disk info
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         202G   34G  168G  17% /
tmpfs            64M     0   64M   0% /dev
shm              25G  8.0K   25G   1% /dev/shm
/dev/root       2.0G  1.1G  849M  57% /usr/sbin/docker-init
/dev/sda1       242G   71G  172G  30% /kaggle/input
tmpfs            26G  4.2M   26G   1% /var/colab
tmpfs            26G     0   26G   0% /proc/acpi
tmpfs            26G     0   26G   0% /proc/scsi
tmpfs            26G     0   26G   0% /sys/firmware


In [29]:
# memory info
!cat /proc/meminfo

MemTotal:       53470760 kB
MemFree:        24669020 kB
MemAvailable:   43566876 kB
Buffers:          137640 kB
Cached:         18826808 kB
SwapCached:            0 kB
Active:          3967788 kB
Inactive:       23734008 kB
Active(anon):       3560 kB
Inactive(anon):  8751616 kB
Active(file):    3964228 kB
Inactive(file): 14982392 kB
Unevictable:           4 kB
Mlocked:               4 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               648 kB
Writeback:             0 kB
AnonPages:       8736460 kB
Mapped:           926540 kB
Shmem:             17824 kB
KReclaimable:     579952 kB
Slab:             702380 kB
SReclaimable:     579952 kB
SUnreclaim:       122428 kB
KernelStack:        8864 kB
PageTables:        45068 kB
SecPageTables:         0 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    26735380 kB
Committed_AS:   14494616 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       82384 kB
VmallocChunk:    

In [30]:
# cpu info
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 8
core id		: 0
cpu cores	: 4
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
addres